In [34]:
import re
import codecs
import numpy as np
import pandas as pd
from sklearn import feature_extraction 
from sklearn import linear_model
from sklearn import pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
df = pd.read_csv('Language Detection.csv')

In [35]:
df.head()

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10337 entries, 0 to 10336
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      10337 non-null  object
 1   Language  10337 non-null  object
dtypes: object(2)
memory usage: 161.6+ KB


In [5]:
df.Language.unique()

array(['English', 'Malayalam', 'Hindi', 'Tamil', 'Portugeese', 'French',
       'Dutch', 'Spanish', 'Greek', 'Russian', 'Danish', 'Italian',
       'Turkish', 'Sweedish', 'Arabic', 'German', 'Kannada'], dtype=object)

In [6]:
x = df.Text
y = df.Language

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
encoded_y = le.fit_transform(y)

In [8]:
le.inverse_transform([3])

array(['English'], dtype=object)

In [9]:
data = []
for text in x:
    text = re.sub(r'[!@#$(),.n"%^*?:;~`0-9\n]', '', text)
    text = re.sub(r'[[]]', ' ', text)
    text = text.lower()
    data.append(text)

C:\Users\shankesh\AppData\Local\Temp\ipykernel_3084\2681050958.py:4: FutureWarning: Possible nested set at position 1
  text = re.sub(r'[[]]', ' ', text)


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x = cv.fit_transform(data).toarray()

In [11]:
xtrain, xtest, ytrain, ytest = train_test_split(x, encoded_y, train_size=0.8, random_state = 12)

In [12]:
xtrain.shape

(8269, 38700)

In [32]:
from sklearn.naive_bayes import MultinomialNB
naive_model = MultinomialNB()
naive_model.fit(xtrain, ytrain)

MultinomialNB()

In [39]:
model = Sequential()
model.add(Dense(500, input_dim=38700, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(17, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Train model
model.fit(xtrain, ytrain, epochs=4)

Epoch 1/4
259/259 [==============================] - 64s 243ms/step - loss: 0.4644 - accuracy: 0.8831
Epoch 2/4
259/259 [==============================] - 63s 244ms/step - loss: 0.0276 - accuracy: 0.9932
Epoch 3/4
259/259 [==============================] - 49s 190ms/step - loss: 0.0097 - accuracy: 0.9972
Epoch 4/4
259/259 [==============================] - 55s 213ms/step - loss: 0.0107 - accuracy: 0.9970


In [14]:
yhat = naive_model.predict(xtest)

In [15]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(ytest, yhat)

In [16]:
accuracy

0.9714700193423598

In [51]:
def prediction(text):
    x = cv.transform([text]).toarray()
    lang = naive_model.predict(x)
    lang = le.inverse_transform(lang)
    print('Naive Bayes Prediction : ',lang[0])
    

In [64]:
from tabnanny import verbose


def prediction2(text):
    x = cv.transform([text]).toarray()
    lang = model.predict(x, verbose = 0)
    lang = np.argmax(lang)
    lang = le.inverse_transform([lang])
    print('ANN Prediction : ',lang[0])

In [65]:
prediction('أحتاج إلى شراء دراجة هوندا بعد حصولي على وظيفة')
prediction2('أحتاج إلى شراء دراجة هوندا بعد حصولي على وظيفة')

Naive Bayes Prediction :  Arabic
ANN Prediction :  Arabic


In [66]:
prediction("это портал знаний на базе сообщества для профессионалов в области аналитики и данных.")
prediction2("это портал знаний на базе сообщества для профессионалов в области аналитики и данных.")

Naive Bayes Prediction :  Russian
ANN Prediction :  Russian


In [67]:
prediction("வணக்கம் ச மாப்ள")
prediction2("வணக்கம் ச மாப்ள")

Naive Bayes Prediction :  Tamil
ANN Prediction :  Tamil


In [68]:
prediction("എനിക്ക് ജോലി കിട്ടിയതിന് ശേഷം എനിക്ക് ഒരു ഹോണ്ട ബൈക്ക് വാങ്ങണം")
prediction2("എനിക്ക് ജോലി കിട്ടിയതിന് ശേഷം എനിക്ക് ഒരു ഹോണ്ട ബൈക്ക് വാങ്ങണം")

Naive Bayes Prediction :  Malayalam
ANN Prediction :  Malayalam


In [69]:
prediction("ನಾನು ಕೆಲಸ ಪಡೆದ ನಂತರ ನಾನು ಹೋಂಡಾ ಬೈಕ್ ಖರೀದಿಸಬೇಕಾಗಿದೆ")
prediction2("ನಾನು ಕೆಲಸ ಪಡೆದ ನಂತರ ನಾನು ಹೋಂಡಾ ಬೈಕ್ ಖರೀದಿಸಬೇಕಾಗಿದೆ")

Naive Bayes Prediction :  Kannada
ANN Prediction :  Kannada


In [70]:
prediction("ik moet een honda-fiets kopen nadat ik een baan heb gekregen")
prediction2("ik moet een honda-fiets kopen nadat ik een baan heb gekregen")

Naive Bayes Prediction :  Dutch
ANN Prediction :  Dutch


In [71]:
prediction("necesito comprar una bicicleta honda después de conseguir un trabajo")
prediction2("necesito comprar una bicicleta honda después de conseguir un trabajo")

Naive Bayes Prediction :  Spanish
ANN Prediction :  Spanish


In [72]:
prediction("Jag måste köpa en hondacykel efter att jag fått jobb")
prediction2("Jag måste köpa en hondacykel efter att jag fått jobb")

Naive Bayes Prediction :  Sweedish
ANN Prediction :  Sweedish


In [73]:
prediction("Ich muss mir ein Honda-Motorrad kaufen, nachdem ich einen Job bekommen habe")
prediction2("Ich muss mir ein Honda-Motorrad kaufen, nachdem ich einen Job bekommen habe")

Naive Bayes Prediction :  German
ANN Prediction :  German


In [74]:
prediction("Πρέπει να αγοράσω ένα ποδήλατο honda αφού βρω δουλειά")
prediction2("Πρέπει να αγοράσω ένα ποδήλατο honda αφού βρω δουλειά")

Naive Bayes Prediction :  Greek
ANN Prediction :  Greek


In [75]:
prediction("eu preciso comprar uma moto honda depois que eu conseguir um emprego")
prediction2("eu preciso comprar uma moto honda depois que eu conseguir um emprego")

Naive Bayes Prediction :  Portugeese
ANN Prediction :  Portugeese
